In [1]:
import os
import pandas as pd
import json
import numpy as np
import pickle
import ast
from fuzzywuzzy import fuzz,process
import functools
import os
pd.set_option('display.max_columns', None)

In [2]:
Ruta_archivos_Yelp = r"G:\Jorge\Desktop\Yelp"

## 1 ETL de Archivos Yelp

### 1.1 Archivo checkin.json

In [3]:
#Funcion para abrir chekin de yelp

def abrir_Archivo_json(archivo):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    with open(archivo) as file:
        for line in file:
            try:
                obj = json.loads(line)
                merged_data.append(obj)
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en el archivo {archivo}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

# Ejemplo de uso
archivo = os.path.join(Ruta_archivos_Yelp, 'checkin.json')
df_checkin_yelp = abrir_Archivo_json(archivo)

In [4]:
# Revisamos si tiene nulos
df_checkin_yelp.isnull().sum()

business_id    0
date           0
dtype: int64

In [5]:
# Revisamos si tiene duplicados
df_checkin_yelp.duplicated().sum()

0

In [6]:
# Desanidamos la fecha (en Fecha y hora)
desanidados = []

for _, row in df_checkin_yelp.iterrows():
    business_id = row['business_id']
    dates = row['date'].split(', ')
    for date in dates:
        time, date = date.split(' ')
        desanidados.append([business_id, time, date])

df_desanidado = pd.DataFrame(desanidados, columns=['business_id', 'hour', 'fecha'])

In [7]:
# Preview
df_desanidado.head(2)

,business_id,hour,fecha
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13,21:10:56
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02,22:18:06


In [8]:
# Exportamos a CSV
df_desanidado.to_csv('Salidas/Dataset_Checkin.csv', index=False)

### 1.2 Archivo user.parquet

In [9]:
# Leemos el archivo
archivo = os.path.join(Ruta_archivos_Yelp, "user.parquet")
df_User_yelp = pd.read_parquet(archivo)

In [10]:
# Eliminamos duplicados y reindexamos
df = df_User_yelp.drop_duplicates(subset=["user_id", "name"], keep="first")
df_User_yelp = df_User_yelp.reindex(df.index)
df_User_yelp['id_user'] = df.index

In [11]:
# Preview
df_User_yelp.head(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,id_user
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91,250,65,55,56,18,232,844,467,467,239,180,0
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,3.74,1145,264,184,157,251,1847,7054,3131,3131,1521,1946,1


In [12]:
# Creamos y exportamos un nuevo dataframe para conservar id_user original con su user_id , el cual será usado para unir tablas
dfusuario = df_User_yelp.loc[:, ["id_user",'user_id']]

In [13]:
#Renombramos la columna user_id
dfusuario.rename(columns={"user_id": "yelp_id"}, inplace=True)

In [14]:
# Preview
dfusuario.head(2)

,id_user,yelp_id
0,0,qVc8ODYU5SZjKXVBgXdI7w
1,1,j14WgRoU_-2ZE1aw1dXrJg


In [15]:
#Exportamos el nuevo dataframe llamado dfusuario
dfusuario.to_csv("Salidas/UsuarioYelp.csv")

In [16]:
# Eliminamos la columna user_id del dataframe df_user_yelp
df_User_yelp.drop(columns=['user_id'], inplace=True)

In [17]:
# Aplicamos transformaciones a los tipos de datos en las diferentes variables

# Cambiamos a string la variable name
df_User_yelp['name'] = df['name'].astype(str)

# Cambiamos a tipo datetime la variable yelping_since
df_User_yelp['yelping_since'] = pd.to_datetime(df['yelping_since'])

# Creamos una columna año a partir de elite que devuelva una lista de años separado por comas
df_User_yelp['years'] = df['elite'].str.split(',')

# Creamos un nuevo DataFrame con filas individuales para cada año y usuario
years_df = df_User_yelp[['id_user', 'years']].explode('years')

# Reemplazamos valores vacios por np.nan en years
years_df['years'] = years_df['years'].replace("", np.nan)

# Reemplazamos 20 po 2020 en years
years_df['years'] = years_df['years'].replace("20", "2020")

In [ ]:
years_df.head(2)

In [18]:
# Exportamos segundo dataframe de user_yelp como usuarioElite
years_df.to_csv("Salidas/Dataset_User_Elite.csv")

In [19]:
# Eliminamos las columnas elite y years del dataframe original user.parquet
df_User_yelp.drop(columns=['elite'], inplace=True)
df_User_yelp.drop(columns=['years'], inplace=True)

## 1.3 Archivo business.pkl

In [20]:
# Leemos el archivo
archivo = os.path.join(Ruta_archivos_Yelp, 'business.pkl')
df_business= pd.read_pickle(archivo)

In [21]:
# Agregamos nuevas columnas al dataframe con valores iniciales en NONE
df_business['NAME']=None
df_business['REVIEW_COUNT']=None
df_business['POSTAL_CODE']=None
df_business['CITY']=None
df_business['STATE']=None
df_business['BUSINESS_ID']=None
df_business['ADDRESS']=None
df_business['LATITUDE']=None
df_business['LONGITUDE']=None
df_business['STARS']=None
df_business['IS_OPEN']=None
df_business['ATTRIBUTES']=None
df_business['CATEGORIES']=None
df_business['HOURS']=None

In [22]:
# Obtenemos la actualización de la columna "NAME" del DataFrame df_business, donde se fusionan las cadenas de texto presentes en cada valor, eliminando los caracteres no-alfabéticos.
for index,i in enumerate(df_business.name.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'NAME'] = ''.join(arr)

In [23]:
# Obtenemos la actualización de la columna "CITY" del DataFrame df_business, donde se fusionan las cadenas de texto presentes en cada valor, eliminando los caracteres no-alfabéticos.
for index,i in enumerate(df_business.city.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'CITY'] = ''.join(arr)

In [24]:
# Obtenemos la actualización de la columna "STATE" del DataFrame df_business, donde se fusionan las cadenas de texto presentes en cada valor, eliminando los caracteres no-alfabéticos.
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'STATE'] = ''.join(arr)

In [25]:
# En este script, se actualiza la columna 'REVIEW_COUNT' del DataFrame df_business con el primer número entero encontrado en cada valor, recorriendo los valores de la columna y almacenando los números enteros en una lista antes de asignarlos a la columna.
for index,i in enumerate(df_business.review_count.values):
    arr=[]
    for e in i:
        if isinstance(e,int):
         arr.append(e)
    df_business.loc[index, 'REVIEW_COUNT'] = arr[0]

In [26]:
# Se fusionan los caracteres de texto presentes en cada valor de la columna 'business_id' del DataFrame df_b, actualizando la columna "BUSINESS_ID" con los valores resultantes
for index,i in enumerate(df_business.business_id.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'BUSINESS_ID'] = ''.join(arr)

In [27]:
# Se fusionan los caracteres de texto presentes en cada valor de la columna 'address' del DataFrame 'df_business', actualizando la columna 'ADDRESS' con los valores resultantes
for index,i in enumerate(df_business.address.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'ADDRESS'] = ''.join(arr)

In [28]:
# Se fusionan los caracteres de texto presentes en cada valor de la columna 'postal_code' del DataFrame 'df_business', actualizando la columna 'POSTAL_CODE' con los valores resultantes
for index,i in enumerate(df_business.postal_code.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'POSTAL_CODE'] = ''.join(arr)

In [29]:
# Se filtran los elementos numéricos mayores que 1 en cada valor de la columna 'latitude' del DataFrame 'df_business'. A continuación, se asigna el primer elemento filtrado a la columna 'LATITUDE' en el DataFrame 'df_business' en la fila correspondiente al índice actual.
for index,i in enumerate(df_business.latitude.values):
    arr=[]
    for e in i:
       if e>1:
         arr.append(e)
    df_business.loc[index, 'LATITUDE'] = arr[0]

In [30]:
# Se filtran los elementos numéricos menores que -1 en cada valor de la columna 'longitude' del DataFrame 'df_business'. Luego, se asigna el primer elemento filtrado a la columna 'LONGITUDE' en el DataFrame 'df_business' en la fila correspondiente al índice actual.
for index,i in enumerate(df_business.longitude.values):
    arr=[]
    for e in i:
        if e<-1:
            arr.append(e)
    df_business.loc[index, 'LONGITUDE'] = arr[0]

In [31]:
# Se filtran los elementos numéricos mayores que 0.1 en cada valor de la columna 'stars' del DataFrame 'df_business'. Después, se asigna el primer elemento filtrado a la columna 'STARS' en el DataFrame 'df_business' en la fila correspondiente al índice actual.
for index,i in enumerate(df_business.stars.values):
    arr=[]
    for e in i:
       if e>0.1:
         arr.append(e)
    df_business.loc[index, 'STARS'] = arr[0]

In [32]:
# Se filtran los elementos numéricos mayores o iguales a 0 en cada valor de la columna 'is_open' del DataFrame 'df_business'. A continuación, se asigna el primer elemento filtrado a la columna 'IS_OPEN' en el DataFrame 'df_business' en la fila correspondiente al índice actual
for index,i in enumerate(df_business.is_open.values):
    arr=[]
    for e in i:
       if e >=0:
         arr.append(e)
    df_business.loc[index, 'IS_OPEN'] = arr[0]

In [ ]:
# Este código busca y guarda el primer diccionario encontrado en la columna 'attributes' del dataframe 'df_business' en la columna 'ATTRIBUTES'.
# No se puede evitar los warnings porque por alguna extraña razón df_business.loc[index,'ATTRIBUTES']=arr[0] no funciona
for index,i in enumerate(df_business.attributes.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
            arr.append(e)
    if len(arr)>0:
        df_business['ATTRIBUTES'][index] = arr[0]

In [34]:
# Este código filtra los elementos de tipo cadena en la columna 'categories' del dataframe 'df_business'. Luego, fusiona todas las cadenas filtradas en una sola cadena y la asigna a la columna 'CATEGORIES' en el dataframe 'df_business'.
for index,i in enumerate(df_business.categories.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business.loc[index, 'CATEGORIES'] = ''.join(arr)

In [ ]:
# En este código, se recorren los valores de la columna 'hours' en el dataframe 'df_business' y se filtran los elementos que son diccionarios. Luego, se asigna el primer diccionario encontrado a la columna 'HOURS' en el dataframe 'df_business' en la fila correspondiente. En resumen, el código extrae y guarda el primer diccionario encontrado en la columna 'hours' del dataframe 'df_business' en la columna 'HOURS'.
# No se puede evitar los warnings porque por alguna extraña razón df_business.loc[index,'HOURS']=arr[0] no funciona
for index,i in enumerate(df_business.hours.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
            arr.append(e)
    if len(arr)>0:     
        df_business['HOURS'][index] = arr[0]

In [36]:
# Filtramos el dataframe solo a las columnas nuevas generadas con los bucles, con los nombres iniciales
df_business=df_business[['BUSINESS_ID','NAME','REVIEW_COUNT','CITY','STATE','ADDRESS','POSTAL_CODE','LATITUDE','LONGITUDE','STARS','IS_OPEN','ATTRIBUTES','CATEGORIES','HOURS']]

In [37]:
df_business.rename(columns = {'BUSINESS_ID':'business_id',
                              'NAME':'name',
                              'ADDRESS':'address',
                              'CITY':'city',
                              'STATE':'state',
                              'POSTAL_CODE':'postal_code',
                              'LATITUDE':'latitude',
                              'LONGITUDE':'longitude',
                              'STARS':'stars',
                              'REVIEW_COUNT':'review_count',
                              'IS_OPEN':'is_open',
                              'ATTRIBUTES':'attributes',
                              'CATEGORIES':'categories',
                              'HOURS':'hours'}, inplace = True)
df_business = df_business[['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours']]

In [38]:
df_business2 = df_business.copy()
df_business2.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."


In [39]:
# Cargamos un nuevo CSV con las ciudades de Estados Unidos
# Fuente: https://simplemaps.com/data/us-cities
city = pd.read_csv('Orig/Propio/uscities.csv')
city.head(2)

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18908608,11080.3,shape,False,True,America/New_York,1,11229 11228 11226 11225 11224 11222 11221 1122...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,11922389,3184.7,shape,False,True,America/Los_Angeles,1,91367 90291 90293 90292 91316 91311 90035 9003...,1840020491


In [40]:
#ciudades unicas de estados unidos
ciudades_estados_unidos = city['city'].unique()

In [41]:
# la primera función busca la mejor coincidencia de una ciudad en una lista, mientras que la segunda función calcula un umbral de similitud basado en la longitud de una ciudad. Ambas funciones se utilizan en conjunto para determinar la mejor coincidencia de una ciudad y aplicar un criterio de aceptación basado en el umbral de similitud.
import functools

@functools.lru_cache(maxsize=None)  
def encontrar_mejor_coincidencia(ciudad):
    mejor_coincidencia = process.extractOne(ciudad, ciudades_estados_unidos)
    resultado = mejor_coincidencia[0] if mejor_coincidencia[1] >= calcular_umbral_similitud(len(ciudad)) else ciudad
    return resultado

def calcular_umbral_similitud(longitud_ciudad):
    umbral_base = 55
    umbral = umbral_base - (longitud_ciudad // 3)
    return max(umbral, umbral_base)

In [42]:
# Probamos la funcion
encontrar_mejor_coincidencia('nw york')

'New York'

In [43]:
# Se aplica la función encontrar_mejor_coincidencia a cada valor de la columna 'city' del DataFrame df_b. La función busca la mejor coincidencia de cada ciudad en una lista de ciudades de EE.UU. y actualiza la columna 'city' con las mejores coincidencias encontradas.
# Demora como 17-23 min
df_business2['city'] = df_business2['city'].apply(encontrar_mejor_coincidencia)

In [44]:
# Generamos un dataframe desde city con las columnas City y State solamente
citystate = city[['city', 'state_name']]

In [45]:
# Se fusionan dos DataFrames ('df_business' y 'citystate') por la columna 'city' en una unión izquierda.
df_business2 = df_business2.merge(citystate, left_on='city', right_on='city', how='left')

In [46]:
# Eliminar la columna duplicada 'state'
df_business2.drop('state', axis=1, inplace=True)

In [47]:
# Se actualiza el dataframe df_business, manteniendo solo las columnas mencionadas.
df_business2 = df_business2[['business_id', 'name', 'address', 'city', 'state_name',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours']]

In [48]:
# Renombramos la columna state_name a state
df_business2 = df_business2.rename(columns={'state_name':'state'})

In [49]:
df_business2.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,California,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,Missouri,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."


In [50]:
# Filtramos el dataframe con los 5 estados que se usarán en el proyecto
# statein = ['New York' , 'Florida' , 'California', 'Nevada' , 'Hawaii']
statein = ['New York' , 'Florida' , 'California', 'Nevada' , 'Hawaii', "Connecticut", "New Jersey", "Texas", "Pennsylvania", "Alaska"]
df_business2 = df_business2[df_business2['state'].isin(statein)].reset_index()

In [51]:
# print error comprobar huwaii

In [52]:
df_business2.head(2)

,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,California,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."


In [53]:
## Renombramos index
df_business2 = df_business2.rename(columns={'index':'id_business'})

In [54]:
# Generamos un nuevo dataframe solo con las columnas 'id_business','business_id'
#Creamos la tabla de dimension BusinessYelp
BusinessYelp = df_business2[['id_business', 'business_id']]

# Renombrar la columna "business_id" como "businessYelp_id"
BusinessYelp.rename(columns={"business_id": "businessYelp_id"}, inplace=True)
BusinessYelp.head(2)

C:\Users\jalvarez\AppData\Local\Temp\ipykernel_11220\3899857145.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BusinessYelp.rename(columns={"business_id": "businessYelp_id"}, inplace=True)


,id_business,businessYelp_id
0,0,Pns2l4eNsfO8kk83dixA6A
1,3,MTSW4McQd7CbVtyjqoe9mw


In [55]:
# Exportamos la tabla
BusinessYelp.to_csv('Salidas/BusinessYelpId.csv',index=False)

In [56]:
# Filtramos por ultima vez el dataframe
df_business2 = df_business2[['id_business','name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours']]
df_business2.head(2)

,id_business,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,0,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,California,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,3,St Honore Pastries,935 Race St,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."


In [57]:
# Convertimos a númerico la columna postal_code
df_business2["postal_code"] = df_business2["postal_code"].apply(pd.to_numeric, errors='coerce')

In [ ]:
print error

In [58]:
# Creamos un dataframe auxiliar  que contiene solo las filas donde el valor de la columna 'state' coincide con "New York", "Florida", "California", "Nevada" o "Hawuai".
aux = df_business2.query('state == "New York" or state == "Florida" or state == "California" or state == "Nevada" or state == "Hawuai"')

In [59]:
# Buscamos filas específicas en el DataFrame y realizamos cambios en las columnas 'state' y 'postal_code' en función de los valores de las columnas 'city' y 'postal_code'.
df_business2.loc[df_business2["city"] == "Saint Petersburg", "city"] = "St. Petersburg"
df_business2.loc[(df_business2["city"] == "St. Petersburg") & (df_business2["postal_code"].isnull()), "state"] = "Florida"
df_business2.loc[(df_business2["city"] == "St. Petersburg") & (df_business2["postal_code"].isnull()), "postal_code"] = 33707
df_business2.loc[(df_business2["city"] == "Tampa") & (df_business2["postal_code"].isnull()), "state"] = "Florida"
df_business2.loc[(df_business2["city"] == "Tampa") & (df_business2["postal_code"].isnull()), "postal_code"] = 33610
df_business2.loc[(df_business2["city"] == "Santa Barbara") & (df_business2["postal_code"].isnull()), "state"] = "California"
df_business2.loc[(df_business2["city"] == "Santa Barbara") & (df_business2["postal_code"].isnull()), "postal_code"] = 93101
df_business2.loc[(df_business2["city"] == "Pasco") & (df_business2["postal_code"].isnull()), "state"] = "Florida"
df_business2.loc[(df_business2["city"] == "Pasco") & (df_business2["postal_code"].isnull()), "postal_code"] = 33544

In [60]:
# Exportamos df_business2 para el proyecto
df_business2.to_csv('Salidas/Dataset_Business.csv',index=False)

In [61]:
# Eliminamos algunas columnas del dataframe auxiliar
aux.drop(columns=['is_open'], inplace=True)
aux.drop(columns=['postal_code'], inplace=True)

C:\Users\jalvarez\AppData\Local\Temp\ipykernel_11220\3192928666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux.drop(columns=['is_open'], inplace=True)
C:\Users\jalvarez\AppData\Local\Temp\ipykernel_11220\3192928666.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux.drop(columns=['postal_code'], inplace=True)


In [62]:
# Eliminamos duplicados de la columna auxiliar y reindexamos
aux = aux.drop_duplicates(subset=["id_business", "name"], keep="first")
aux['id_business'] = aux.index
aux.head(2)

,id_business,name,address,city,state,latitude,longitude,stars,review_count,attributes,categories,hours
0,0,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,California,34.426679,-119.711197,5.0,7,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
3,3,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,New York,38.627695,-90.340465,2.5,13,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."


In [63]:
# Generamos un nuevo dataframe con "id_business",'categories'

#Tabla de dimension de categorias
df_cat = aux.loc[:, ['id_business','categories']]

In [64]:
# Dividir los valores de la columna "categories" por coma y expandirlos en filas
df_cat['categories'] = df_cat['categories'].str.split(',')
df_cat = df_cat[['id_business', 'categories']].explode('categories')

In [65]:
# Creamos un array con las categorias unicas
categorias = df_cat["categories"].unique()
# Creamos la tabla de dimension de categorias de yelp
df_categorias = pd.DataFrame(categorias, columns=["Descripcion"])
df_categorias['IdCategoria'] = df_categorias.index

In [66]:
# Exportamos la tabla de dimensiones de categorias
df_categorias.to_csv("Salidas/BusinessCategorias.csv")

In [67]:
# Creamos una nuevo dataframe auxiliar para el detalle de las categorias a partir de df_Cat y df_categorias
df_aux = df_cat.merge(df_categorias, left_on="categories", right_on="Descripcion", how="inner")

In [68]:
# Eliminamos columnas categories y Descripcion para generar una tabla de dimension entre id_business y idcategoria
df_aux.drop(columns=["categories"],inplace=True)
df_aux.drop(columns=["Descripcion"],inplace=True)

In [69]:
# Exportamos tabla de dimensiones de detalle de categorias
df_aux.to_csv("Salidas/BusinessDetalleCategorias.csv")

In [70]:
# Eliminamos categories de aux
aux.drop(columns=["categories"],inplace=True)

In [71]:
# Generamos tabla de dimension atributos a partir de aux
df_atribute = aux.loc[:, ["id_business",'attributes']]

In [72]:
# Procesamos y dividimos los elementos de la columna 'attributes'
for index, i in enumerate(df_atribute['attributes']):
    if isinstance(i,str):
       df_atribute['attributes'][index] = i[1:-1].split(',')

In [73]:
# Expandimos la columna 'attributes' del DataFrame 'df' en filas separadas, manteniendo los valores correspondientes de la columna 'id_business'.
df_atribute = df_atribute[['id_business', 'attributes']].explode('attributes')

In [74]:
# Eliminamos los corchetes y comillas dobles de la columna 'attributes' del DataFrame, dejando los valores limpios y sin esos caracteres específicos
df_atribute['attributes'] = df_atribute['attributes'].str.replace('{', '').str.replace('}', '')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('"', '')

In [75]:
# Modificamos los valores en la columna 'attributes', actualizando ciertos aspectos relacionados con estacionamiento de negocios
df_atribute['attributes'] = df_atribute['attributes'].str.replace('street: True', 'BusinessParking street: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('street: False', 'BusinessParking street: False')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('validated: True', 'BusinessParking validated: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('validated: False', 'BusinessParking validated: False')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('lot: True', 'BusinessParking lot: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('lot: False', 'BusinessParking lot: False')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('valet: True', 'BusinessParking valet: True')
df_atribute['attributes'] = df_atribute['attributes'].str.replace('valet: False', 'BusinessParking valet: False')

In [76]:
# Creamos un nuevo DataFrame llamado 'df_atributos' que contiene los valores únicos de la columna 'attributes' del DataFrame original 'df_atribute', junto con una columna de identificación única para cada valor.
atributos = df_atribute["attributes"].unique()
df_atributos = pd.DataFrame(atributos, columns=["Descripcion"])
df_atributos['IdAtributos'] = df_atributos.index

In [77]:
# Exportamos tabla de atributos
df_atributos.to_csv("Salidas/BusinessAtributos.csv")

In [78]:
# Eliminamos las columnas "attributes" de aux
aux.drop(columns=["attributes"],inplace=True)

In [79]:
# Creamos la tabla de dimensiones "Hours" a partir de aux
df_hours = aux.loc[:, ["id_business",'hours']]
df_hours

,id_business,hours
0,0,None
3,3,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
4,4,"{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
6,6,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
8,8,"{'Monday': '9:30-21:30', 'Tuesday': '9:30-21:3..."
...,...,...
124740,124740,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-20:0', '..."
124747,124747,"{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'..."
124751,124751,"{'Monday': '9:30-18:0', 'Tuesday': '9:30-18:0'..."
124753,124753,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [80]:
# Exportamos la tabla de Hours
df_hours.to_csv("Salidas/BusinessHoras.csv")

In [81]:
# Creamos el dataframe df_Hour_detalle
df_hour_detalle = df_business2[['id_business','hours']]

In [82]:
df_hour_detalle = df_hour_detalle.explode('hours')
df_hour_detalle

,id_business,hours
0,0,None
1,3,Monday
1,3,Tuesday
1,3,Wednesday
1,3,Thursday
...,...,...
124756,528135,Tuesday
124756,528135,Wednesday
124756,528135,Thursday
124756,528135,Friday


In [83]:
# Creamos un dataframe llamado hour con las horas unicas de df_hour_detalle
hours = df_hour_detalle['hours'].unique()

In [84]:
# Se crea un nuevo DataFrame que contiene una sola columna llamada "Descripcion" que contiene los valores de la columna 'hours' y reindexamos
DescHour = pd.DataFrame(hours,columns=["Descripcion"])
DescHour['id_hour'] = DescHour.index

In [85]:
# Exportamos el dataframe BusinessHorarios
DescHour.to_csv('Salidas/BusinessHorarios.csv',index=False)

In [86]:
# Hacemos una union entre df_hour_detalle y DescHour
df_hour_detalle.merge(DescHour, left_on='hours', right_on='Descripcion', how='inner')[['id_business','id_hour']]

,id_business,id_hour
0,0,0
1,3,1
2,3,2
3,3,3
4,3,4
...,...,...
685371,528135,2
685372,528135,3
685373,528135,4
685374,528135,5


In [87]:
# Exportamos el dataframe df_hour_detalle
df_hour_detalle.to_csv('Salidas/BusinessDetallesHora.csv',index=False)

In [88]:
# Creamos el dataframe df_detalleatributo a partir de la union entre df_atribute y df_atributos
df_detalleatributo = df_atribute.merge(df_atributos, left_on="attributes", right_on="Descripcion", how="inner")

In [89]:
# Eliminamos las columnas "attributes" y "Descripcion" de df_detalleatributo
df_detalleatributo.drop(columns = "attributes", inplace=True)
df_detalleatributo.drop(columns = "Descripcion", inplace=True)

In [90]:
# Exportamos dataframe df_detalleatributo
df_detalleatributo.to_csv("Salidas/BusinessDetalleAtributos.csv")

In [91]:
# Eliminamos la columna hours de aux
aux.drop(columns=["hours"],inplace=True)
aux.head(2)

,id_business,name,address,city,state,latitude,longitude,stars,review_count
0,0,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,California,34.426679,-119.711197,5.0,7
3,3,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,New York,38.627695,-90.340465,2.5,13


## 1.4 tip.json

In [92]:
# Abrimos el archivo json
json_objects=[]

archivo = os.path.join(Ruta_archivos_Yelp, 'tip.json')
with open(archivo, 'r',encoding='utf-8') as f:
    for line in f:
        json_objects.append(json.loads(line))

df_tip = pd.DataFrame(json_objects)

In [93]:
# Obtenemos un DataFrame que contenga solo las filas correspondientes a los negocios presentes en BusinessYelp
df_tip = df_tip[df_tip['business_id'].isin(BusinessYelp.businessYelp_id.unique().tolist())]
df_tip

,user_id,business_id,text,date,compliment_count
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
6,SMGAlRjyfuYu-c-22zIyOg,kH-0iXqkL7b8UXNpguBMKg,"Saturday, Dec 7th 2013, ride Patco's Silver Sl...",2013-12-03 23:42:15,0
7,YVBB9g23nuVJ0u44zK0pSA,jtri188kuhe_AuEOJ51U_A,This is probably the best place in the cool Sp...,2016-11-22 22:14:58,0
...,...,...,...,...,...
908908,5hJR7ljJbhFgOaLi8iz5pQ,AXC__4yZrn-N3BT7-2bV_Q,"BOMB food. Super delicious, great outdoor spac...",2021-05-04 21:44:53,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [94]:
# Hacemos una unión df_tip con BusinessYelp
df_tip = BusinessYelp.merge(df_tip, left_on='businessYelp_id', right_on='business_id', how='right')[['user_id', 'id_business', 'text', 'date', 'compliment_count']]

In [95]:
# Hacemos una unión df_tip con UsuarioYelp
df_tip = dfusuario.merge(df_tip, left_on='yelp_id', right_on='user_id', how='right')[['id_user', 'id_business', 'text', 'date', 'compliment_count']]

In [108]:
# Exportamos dataframe tip.json
df_tip.to_csv('Salidas/Dataset_Tip.csv', index=False)

## 1.5 review.json

In [97]:
# Leemos el archivo review.json de Yelp
jsonarr=[]
archivo = os.path.join(Ruta_archivos_Yelp, "review.json")
with open(archivo, 'r',encoding='utf-8') as f:
    for line in f:
        jsonarr.append(json.loads(line))

df_review = pd.DataFrame(jsonarr)

In [98]:
# Eliminamos duplicados y reindexamos
df_review = df_review.drop_duplicates(subset=["review_id", "text"], keep="first")
df_review['id_review'] = df_review.index

In [99]:
# Creamos un dataframe con "id_review", "review_id" llamado df_ReviewYelp
df_ReviewYelp = df_review[["id_review", "review_id"]]

In [100]:
# Exportamos el dataframe ReviewYelpId
df_ReviewYelp.to_csv("Salidas/ReviewYelpId.csv")

In [101]:
# Eliminamos columna review_id de df_Review
df_review.drop(columns = "review_id",inplace=True)

In [102]:
# Unimos el dataframe df_review con dfusuario
df_review = df_review.merge(dfusuario, left_on="user_id", right_on="yelp_id", how="inner")

In [103]:
# Creamos el dataframe df_reviewfinal a partir de df_review y BusinessYelp
df_reviewfinal = df_review.merge(BusinessYelp, left_on="business_id", right_on="businessYelp_id", how="inner")

In [104]:
# Se eliminan las columnas "user_id" y "yelp_id" del dataframe df_reviewfinal
df_reviewfinal.drop(columns = "business_id", inplace=True)
df_reviewfinal.drop(columns = "businessYelp_id", inplace=True)
df_reviewfinal.drop(columns = "yelp_id", inplace=True)

In [105]:
df_reviewfinal.head(2)

,user_id,stars,useful,funny,cool,text,date,id_review,id_user,id_business
0,mh_-eMZ6K5RLWhZyISBhwA,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,0,124234,6274
1,OyoGAe7OKpv6SyGZT5g77Q,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,1,51327,44841
2,_7bHUi9Uuf5__HHc_Q8guQ,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,3,164430,21993


In [107]:
# Exportamos dataframe review
df_reviewfinal.to_csv("Salidas/Dataset_Review.csv")

## Extra Convert to parquet

In [9]:
import pyarrow.csv as pv
import pyarrow.parquet as pq
import pandas as pd

In [10]:
table = pv.read_csv("Salidas/Dataset_Business.csv")
pq.write_table(table, "Salidas/Dataset_Business.parquet")

In [11]:
table = pv.read_csv("Salidas/Dataset_Checkin.csv")
pq.write_table(table, "Salidas/Dataset_Checkin.parquet")

In [15]:
table = pd.read_csv("Salidas/Dataset_Review.csv")
table.to_parquet("Salidas/Dataset_Review.parquet")

In [16]:
table = pd.read_csv("Salidas/Dataset_Tip.csv")
table.to_parquet("Salidas/Dataset_Tip.parquet")

In [14]:
table = pv.read_csv("Salidas/Dataset_User_Elite.csv")
pq.write_table(table, "Salidas/Dataset_User_Elite.parquet")